In [1]:
import torch
from matplotlib import pyplot as plt
import matplotlib.patches as patches

In [5]:
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', model='nvidia_ssd')
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', model='nvidia_ssd_processing_utils')

Using cache found in C:\Users\ASUS/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub
Using cache found in C:\Users\ASUS/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


ModuleNotFoundError: No module named 'skimage'

In [6]:
ssd_model.to('cuda')
ssd_model.eval()

AssertionError: Torch not compiled with CUDA enabled

In [4]:
uris = [
    'http://images.cocodataset.org/val2017/000000397133.jpg',
    'http://images.cocodataset.org/val2017/000000037777.jpg',
    'http://images.cocodataset.org/val2017/000000252219.jpg'
]

D:\Utilisateurs\anaconda3\envs\new\lib\site-packages\transformers\models\detr\modeling_detr.py:422: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.embedding_dim)


In [5]:
inputs = [utils.prepare_input(uri) for uri in uris]
tensor = utils.prepare_tensor(inputs)

In [8]:
with torch.no_grad():
    detections_batch = ssd_model(tensor)

tensor([[[0.4433, 0.5302, 0.8853, 0.9056],
         [0.5494, 0.2517, 0.0529, 0.2015],
         [0.4998, 0.5360, 0.9956, 0.9024],
         [0.5534, 0.1835, 0.0415, 0.0616],
         [0.4998, 0.4116, 0.9948, 0.8237],
         [0.5957, 0.5033, 0.8091, 0.9647],
         [0.5416, 0.2791, 0.0393, 0.2099],
         [0.1607, 0.2160, 0.2274, 0.1434],
         [0.1738, 0.1822, 0.1956, 0.0785],
         [0.4997, 0.3887, 0.9994, 0.7828],
         [0.6100, 0.5136, 0.7813, 0.9454],
         [0.4998, 0.5484, 0.9999, 0.8796],
         [0.7717, 0.4983, 0.4555, 0.8899],
         [0.8818, 0.6115, 0.2329, 0.7410],
         [0.5729, 0.5916, 0.8472, 0.7972],
         [0.1542, 0.1895, 0.2465, 0.1191],
         [0.1416, 0.1758, 0.2823, 0.1619],
         [0.7558, 0.4970, 0.4818, 0.9796],
         [0.7421, 0.5181, 0.5235, 0.9266],
         [0.4997, 0.3921, 0.9994, 0.7880],
         [0.6063, 0.5336, 0.7868, 0.9010],
         [0.7650, 0.4147, 0.4705, 0.7395],
         [0.4997, 0.4985, 1.0000, 0.9790],
         [0

In [ ]:
results_per_input = utils.decode_results(detections_batch)
best_results_per_input = [utils.pick_best(results, 0.40) for results in results_per_input]

In [ ]:
classes_to_labels = utils.get_coco_object_dictionary()

In [ ]:
for image_idx in range(len(best_results_per_input)):
    fig, ax = plt.subplots(1)
    # Show original, denormalized image...
    image = inputs[image_idx] / 2 + 0.5
    ax.imshow(image)
    # ...with detections
    bboxes, classes, confidences = best_results_per_input[image_idx]
    for idx in range(len(bboxes)):
        left, bot, right, top = bboxes[idx]
        x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
        rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(x, y, "{} {:.0f}%".format(classes_to_labels[classes[idx] - 1], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))
plt.show()